<a href="https://colab.research.google.com/github/samwil56/earth-analytics-python-env/blob/main/Species_Data_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Packages

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


Import necessary data

In [ ]:
#height_AIM = pd.read_csv('/content/drive/MyDrive/thesis_exploration/AIM_data_extraction/height_data_AIM.csv',low_memory=False)
lpi_AIM = pd.read_csv('/content/drive/MyDrive/thesis_exploration/AIM_data_extraction/lpi_aim.csv',low_memory=False)
full_census_AIM = pd.read_csv('/content/drive/MyDrive/thesis_exploration/AIM_data_extraction/AIM_census.csv',low_memory=False)
species_id_AIM = pd.read_csv('/content/drive/MyDrive/thesis_exploration/AIM_data_extraction/species_id_AIM.csv',low_memory=False)

#height_LMF = pd.read_csv('/content/drive/MyDrive/thesis_exploration/AIM_data_extraction/height_LMF.csv',low_memory=False)
lpi_LMF = pd.read_csv('/content/drive/MyDrive/thesis_exploration/AIM_data_extraction/lpi_LMF.csv', low_memory=False)
full_census_LMF = pd.read_csv('/content/drive/MyDrive/thesis_exploration/AIM_data_extraction/si_LMF.csv', low_memory=False)
species_id_LMF = pd.read_csv('/content/drive/MyDrive/thesis_exploration/AIM_data_extraction/species_id_LMF.csv', low_memory=False)

#This is Rachel's csv from 2016
original_list = pd.read_csv('/content/drive/MyDrive/thesis_exploration/AIM_data_extraction/Rachel_PlantList_FINAL.csv', low_memory=False, encoding='windows-1252')

This chunk combines the 'descriptor' csv's provided for AIM and LMF data. There are no observations in these CSV's

In [ ]:
frames = [species_id_LMF,species_id_AIM]
species_id = pd.concat(frames)
species_id = species_id.drop_duplicates(subset='Species')

#print(species_id)

For all AIM data, get counts on number of times each unique species code occurs in LPI and Species Richness

In [ ]:
species = pd.DataFrame()


# get unique codes for all places that species are recorded in AIM
hit1_AIM = list(lpi_AIM.TopCanopy.unique())
hit2_AIM = list(lpi_AIM.Lower1.unique())
hit3_AIM = list(lpi_AIM.Lower2.unique())
hit4_AIM = list(lpi_AIM.Lower3.unique())
hit5_AIM = list(lpi_AIM.Lower4.unique())
basal_AIM = list(lpi_AIM.SoilSurface.unique())
census_AIM = list(full_census_AIM.SpeciesList.unique())

# unique codes for all LPI hits in AIM
lpi_AIM = list(lpi_AIM.TopCanopy) + list(lpi_AIM.Lower1) + list(lpi_AIM.Lower2) + list(lpi_AIM.Lower3) + list(lpi_AIM.Lower4) 

lpi_AIM_pd = pd.DataFrame()
lpi_AIM_pd['specs'] = lpi_AIM + census_AIM
unique_lpi_AIM = lpi_AIM_pd.specs.unique()
AIM = pd.DataFrame()
AIM['AIM.CPLANT'] = unique_lpi_AIM
AIM['AIM.lpi.freq'] = np.nan
AIM['AIM.cen.freq'] = np.nan
x = 0
for i in AIM['AIM.CPLANT']:
    AIM.iloc[[x],[1]] = lpi_AIM.count(i)
    AIM.iloc[[x],[2]] = list(full_census_AIM.SpeciesList).count(i)
    x = x+1



For all LMF data, get counts on number of times each unique species code occurs in LPI and Species Richness
  - worth noting that LMF includes 6 hit + basal for LPI data whereas AIM data only includes 5

In [ ]:
# get unique codes for all places that species are recored in LMF
hit1_LMF = list(lpi_LMF.HIT1.unique())
hit2_LMF = list(lpi_LMF.HIT2.unique())
hit3_LMF = list(lpi_LMF.HIT3.unique())
hit4_LMF = list(lpi_LMF.HIT4.unique())
hit5_LMF = list(lpi_LMF.HIT5.unique())
hit6_LMF = list(lpi_LMF.HIT6.unique())
basal_LMF = list(lpi_LMF.BASAL.unique())
census_LMF = list(full_census_LMF.CPLANT.unique())


# unique codes for all LPI hits in LMF
lpi_LMF = list(lpi_LMF.HIT1) + list(lpi_LMF.HIT2) + list(lpi_LMF.HIT3) + list(lpi_LMF.HIT4) + list(lpi_LMF.HIT5) + list(lpi_LMF.HIT6)
lpi_LMF_pd = pd.DataFrame()
lpi_LMF_pd['specs'] = lpi_LMF + census_LMF
unique_lpi_LMF = lpi_LMF_pd.specs.unique()
LMF = pd.DataFrame()
LMF['LMF.CPLANT'] = unique_lpi_LMF
LMF['LMF.lpi.freq'] = np.nan
LMF['LMF.cen.freq'] = np.nan
x = int(0)
for i in LMF['LMF.CPLANT']:
    LMF.iloc[[x],[1]] = lpi_LMF.count(i)
    LMF.iloc[[x],[2]] = list(full_census_LMF['CPLANT']).count(i)
    x = x+1


Merge the two dataframes with frequency information

In [ ]:
# now merge the LMF and the AIM data
hit_count = LMF.merge(AIM, left_on ='LMF.CPLANT',
                      right_on = 'AIM.CPLANT',
                      how = 'outer')
#print(hit_count.tail())

In order to merge this new dataframe with frequencies back to the original, need to create a column with all species recorded in frequencies for both AIM and LMF

In [ ]:
from pandas.core.ops.array_ops import na_logical_op
hit_count['LMF.CPLANT'] = hit_count['LMF.CPLANT'].fillna(0)
hit_count['AIM.CPLANT'] = hit_count['AIM.CPLANT'].fillna(0)
x=0
hit_count['ALL.SPECIES'] = np.nan

for i in hit_count['LMF.CPLANT']:
  num = hit_count.iat[x,0]
  if num == 0:
    hit_count.iloc[[x],[6]] = hit_count.iloc[[x],[3]]
  else:
    hit_count.iloc[[x],[6]] = hit_count.iloc[[x],[0]]
  x = x+1

A test to make sure it looks correct

In [ ]:
census_LMF = pd.DataFrame(census_LMF)
census_LMF.to_csv('LMF.csv')

Merge the observational data to the species_id dataframe that holds species descriptions

In [ ]:
# all of the unique code information combined 
all_species = hit1_AIM + hit2_AIM + hit3_AIM + hit4_AIM + hit5_AIM + basal_AIM + census_AIM + hit1_LMF + hit2_LMF + hit3_LMF + hit4_LMF + hit5_LMF + hit6_LMF + basal_LMF + hplant_LMF + wplant_LMF + census_LMF

# add that data into a pd df
species['spec'] = all_species
# get the unique codes again in case of overlap
uni = list(species.spec.unique())
# I think this is innefficient and can be cleaned up in the future
species_unique = pd.DataFrame()
species_unique['species'] = uni

# merge this with species information
species_unique1 = species_unique.merge(species_id, left_on = 'species',
                                      right_on = 'Species',
                                      how = 'right')



Merge the species descriptions dataframe to the frequency dataframe. Merge this new dataframe to the csv provided from Rachel. Select the columns of interest and export.

In [ ]:
# export
#species_unique1.to_csv('unique_species.csv', encoding = 'utf-8')

# Since there are versions for each state, drop all the duplicates             


df = species_unique1.merge(hit_count, left_on = 'species',right_on = 'ALL.SPECIES',how='right')

# export
#df.to_csv('unique_species_code.csv', encoding = 'utf-8')

df = df.merge(original_list, left_on = 'ALL.SPECIES', right_on = 'CPLANT',how='outer')
df.to_csv('almost_all.csv', encoding = 'utf-8')

full_df = df[['ALL.SPECIES', 
             'GrowthHabit_y',
             'GrowthHabitSub_y',
             'Duration_y',
             'Noxious',
             'SpeciesState',
             'LMF.lpi.freq',
             'LMF.cen.freq',
             'AIM.lpi.freq',
             'AIM.cen.freq',
             'ScientificName',
             'Family_y',
             'invasive',
             'introduced',
             'Warm',
             'Cool',
             'Rhiz',
             'Bunch',
             'Code'
]]

#full_df is the csv with all species observed with frequencies and Rachel's data!
full_df.to_csv('full_species_list.csv', encoding='utf-8')

In [ ]:
#og = pd.read_csv('/content/drive/MyDrive/AIM_LMF_SpeciesData/full_species_list.csv')
og = full_df
new = pd.read_csv('/content/drive/MyDrive/AIM_LMF_SpeciesData/USDAPlants_lists/USDAPlantsList_FinalMerged.csv')

In [ ]:
df = og.merge(new,left_on='ALL.SPECIES',right_on='Synonym Symbol',how='left')

In [ ]:
print(df.columns)

In [ ]:
df['GrowthHabit_sub'] = np.nan

In [ ]:
x=0
for i in df['ALL.SPECIES']:
  forb = df.iat[x,29]
  grass = df.iat[x,30]
  lichen =df.iat[x,31]
  nonv = df.iat[x,32]
  shrub = df.iat[x,33]
  subshrub = df.iat[x,34]
  tree = df.iat[x,35]
  vine = df.iat[x,36]
  if forb == 1:
    df.iloc[[x],[38]] = 'Forb/herb'
  elif grass == 1:
    df.iloc[[x],[38]] = 'Graminoid'
  elif lichen == 1:
    df.iloc[[x],[38]] = 'Forb/herb'
  elif nonv == 1:
    df.iloc[[x],[38]] = 'Forb/herb'
  elif subshrub == 1:
    df.iloc[[x],[38]] = 'Subshrub'
  elif shrub == 1:
    df.iloc[[x],[38]] = 'Shrub'
  elif tree == 1:
    df.iloc[[x],[38]] = 'Tree'
  elif vine == 1:
    df.iloc[[x],[38]] = 'Forb/herb'
  else:
    df.iloc[[x],[38]] = df.iat[x,3]
  x=x+1

In [ ]:
df['GrowthHabit'] = np.nan

In [ ]:
x=0
for i in df['ALL.SPECIES']:
  growth = df.iat[x,38]
  if growth == 'Shrub' or growth == 'Tree':
    df.iloc[[x],[39]]= 'Woody'
  elif pd.notna(growth) == True:
    df.iloc[[x],[39]] = 'Non-woody'
  else:
    df.iloc[[x],[39]] = df.iat[x,2]
  x=x+1

In [ ]:
df['Duration'] = np.nan

In [ ]:
x=0
for i in df['ALL.SPECIES']:
  ann = df.iat[x,26]
  per =df.iat[x,28]
  bi = df.iat[x,27]
  if ann == 1:
    df.iloc[[x],[40]] = 'Annual'
  elif per == 1:
    df.iloc[[x],[40]] = 'Perennial'
  elif bi == 1:
    df.iloc[[x],[40]] = 'Biennial'
  else:
    df.iloc[[x],[40]] = df.iat[x,4]
  x=x+1

In [ ]:
full_df = df[['ALL.SPECIES', 
             'GrowthHabit',
             'GrowthHabit_sub',
             'Duration',
             'SpeciesState',
             'LMF.lpi.freq',
             'LMF.cen.freq',
             'AIM.lpi.freq',
             'AIM.cen.freq',
             'ScientificName',
             'Family_y',
             'Noxious',
             'Invasive',
             'introduced',
             'Warm',
             'Cool',
             'Rhiz',
             'Bunch',
             'Code'
]]

#full_df is the csv with all species observed with frequencies and Rachel's data!
full_df.to_csv('plant_list_with_USDA.csv', encoding='utf-8')